In [1]:
from PIL import Image, ImageDraw, ImageFilter
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.pyplot as plt
import numpy as np
from rasterize import Scanline_Rasterize_Polygon, Sort_Vertices, Bounding_Box, Raster_Centroid, Raster_BBox, Weighted_Raster_Centroid

In [2]:
def center_rectangle(x,y,l,w,c):
    l = l/2
    w = w/2
    draw.rectangle([x - w, y - l, x + w, y + l],fill=c)

def center_ellipse(x,y,r,c):

    draw.ellipse([x - r, y - r, x + r, y + r],fill=c)


def Outline_Poly(polygon,color,width):
    for i, point in enumerate(polygon):

        k = i + 1
        if k == len(polygon):
            k = 0
        point2 = polygon[k]

        draw.line((point[0],point[1],point2[0],point2[1]),fill=color, width=width)

In [3]:
image_resolution = 500
res = image_resolution
scale_factor = image_resolution / res
img = Image.new('RGB', (image_resolution, image_resolution))
draw = ImageDraw.Draw(img)


In [4]:
def Weighted_Raster_Centroid(pixels, image_array):
    cx = 0
    cy = 0
    total_weight = 0
    pixel_count = len(pixels)
    increment_size = round(pixel_count / 500)
    if increment_size < 1:
        increment_size = 1
    i = 0
    while i < pixel_count:
        pixel = pixels[i]
        # sample_x = int(pixel[1])
        # sample_y = int(np.floor(pixel[0]))
        # sampled_value = image.getpixel((sample_x,sample_y))
        # sampled_value = image_array[sample_x][sample_y]
        # if sampled_value > 0.9:
        #     sampled_value = 1
        # weight = 1 - np.mean(sampled_value) / 255
        # weight = 1 - sampled_value
        weight = 1

        total_weight += weight
        
        cx += pixel[0] * weight
        cy += pixel[1] * weight

        # i += increment_size
        i += 1
    
    # if pixel_count == 0:
    #     return False
    if total_weight == 0:
        total_weight = pixel_count

    if pixel_count == 0 or cx == 0 or cy == 0:
        return False

    cx /= total_weight
    cy /= total_weight


    return [cx, cy]

In [5]:
polygon = np.array([[10,10],[10,20],[20,20],[20,10]])
bbox = Raster_BBox(polygon, image_resolution)
rasterized = Scanline_Rasterize_Polygon(polygon,bbox,image_resolution)
print(bbox)

([10, 10], [20, 20])


In [6]:
scale_factor = 10
pixel_count = 0
img_width = int(image_resolution / scale_factor)
offset = 0
for x in range(img_width):
    for y in range(img_width):
        center_rectangle((x + offset)*scale_factor, (y + offset)*scale_factor, scale_factor*.8, scale_factor*.8,'white')
offset = 0
for pixel in rasterized:
    x = pixel[0]
    y = pixel[1]
    center_rectangle((x + offset)*scale_factor, (y + offset)*scale_factor, scale_factor*.8, scale_factor*.8,'red')

for point in polygon:
    x = point[0]
    y = point[1]
    center_ellipse((x + offset)*scale_factor, (y + offset)*scale_factor, scale_factor / 2,'blue')

for point in bbox:
    x = point[0]
    y = point[1]
    center_ellipse((x + offset)*scale_factor, (y + offset)*scale_factor, scale_factor / 4,'black')

print(img_width)
print(50**2)
img.save('centroid_debug.png')

50
2500
